In [1]:
# Install necessary packages if not already installed
!pip install openpyxl emoji transformers datasets

import pandas as pd
import numpy as np
import re
import emoji
from transformers import AutoTokenizer


/cta/users/tuna/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
# Load the dataset
df = pd.read_excel("./Raw-Dataset/riberiioo_tweets.xlsx")
print(f"Initial dataset shape: {df.shape}")
df.head()


Initial dataset shape: (11197, 11)


,CREATED_AT,TEXT,USER_NAME,USER_SCREEN_NAME,DETECTED_SOURCE_LANGUAGE,USER_COUNTRY_DETECTION,USER_FOLLOWERS_COUNT,USER_FRIENDS_COUNT,RETWEET_COUNT,QUOTE_COUNT,TWEETLINK
0,"24/11/2022 13:44:24,000000000",@superlig @Besiktas Neden 8 abi neden. Yapıştı...,Riberio,riberiioo,tr,UNKNOWN,9985,3209,0,2,https://twitter.com/riberiioo/status/159577534...
1,"29/11/2022 21:08:11,000000000",Sizce Tareminin son dakikada ki pozisyonu pena...,Riberio,riberiioo,tr,UNKNOWN,9985,3209,0,0,https://twitter.com/riberiioo/status/159769896...
2,"25/11/2022 20:42:05,000000000",Anti İngiltereciler beğensin. Sayımızı bilelim...,Riberio,riberiioo,tr,UNKNOWN,9985,3209,0,3,https://twitter.com/riberiioo/status/159624284...
3,"28/11/2022 20:54:26,000000000",Torreirasız maç kazanmayı bekliyor adam. Bu ma...,Riberio,riberiioo,tr,UNKNOWN,9985,3209,0,0,https://twitter.com/riberiioo/status/159733311...
4,"05/12/2022 13:44:16,000000000",@ukarakullukcu Bu adam kim?,Riberio,riberiioo,tr,UNKNOWN,9985,3209,0,0,https://twitter.com/riberiioo/status/159976157...


In [21]:
# Drop rows where 'DETECTED_SOURCE_LANGUAGE' is missing
df = df.dropna(subset=['DETECTED_SOURCE_LANGUAGE'])
# Filter for Turkish language tweets
df = df[df['DETECTED_SOURCE_LANGUAGE'] == 'tr']
print(f"Dataset shape after filtering for Turkish language: {df.shape}")


Dataset shape after filtering for Turkish language: (10648, 11)


In [22]:
df = df.drop(columns=['USER_COUNTRY_DETECTION', 'USER_SCREEN_NAME'])


In [24]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags (#hashtag)
    text = re.sub(r'#\w+', '', text)
    # Remove punctuation and numbers, keep Turkish characters
    text = re.sub(r'[^a-zA-ZçÇğĞıİöÖşŞüÜ\s]', '', text)
    # Remove additional symbols {, [, ;, ', ", :
    text = re.sub(r'[\{\[\;\'\"\}:]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply the cleaning function
df['CLEANED_TEXT'] = df['TEXT'].apply(clean_text)
df[['TEXT', 'CLEANED_TEXT']].head()

,TEXT,CLEANED_TEXT
0,@superlig @Besiktas Neden 8 abi neden. Yapıştı...,neden abi neden yapıştı üzerimize ten beri beş...
1,Sizce Tareminin son dakikada ki pozisyonu pena...,sizce tareminin son dakikada ki pozisyonu pena...
2,Anti İngiltereciler beğensin. Sayımızı bilelim...,anti i̇ngiltereciler beğensin sayımızı bilelim
3,Torreirasız maç kazanmayı bekliyor adam. Bu ma...,torreirasız maç kazanmayı bekliyor adam bu maç...
4,@ukarakullukcu Bu adam kim?,bu adam kim


In [25]:
# Remove rows with empty 'CLEANED_TEXT'
df = df[df['CLEANED_TEXT'].str.strip() != '']


In [26]:
def lowercase_turkish(text):
    # Handle Turkish-specific lowercase conversion
    mapping = str.maketrans('Iİ', 'ıi')
    return text.translate(mapping).lower()

df['CLEANED_TEXT'] = df['CLEANED_TEXT'].apply(lambda x: x.encode('utf-8').decode('utf-8'))
df['CLEANED_TEXT'] = df['CLEANED_TEXT'].apply(lowercase_turkish)


In [27]:
# Save the cleaned data for future use
df.to_csv('cleaned_tweets.csv', index=False)

In [2]:
df = pd.read_csv('cleaned_tweets.csv')
df.head()

,CREATED_AT,TEXT,USER_NAME,DETECTED_SOURCE_LANGUAGE,USER_FOLLOWERS_COUNT,USER_FRIENDS_COUNT,RETWEET_COUNT,QUOTE_COUNT,TWEETLINK,CLEANED_TEXT
0,"24/11/2022 13:44:24,000000000",@superlig @Besiktas Neden 8 abi neden. Yapıştı...,Riberio,tr,9985,3209,0,2,https://twitter.com/riberiioo/status/159577534...,neden abi neden yapıştı üzerimize ten beri beş...
1,"29/11/2022 21:08:11,000000000",Sizce Tareminin son dakikada ki pozisyonu pena...,Riberio,tr,9985,3209,0,0,https://twitter.com/riberiioo/status/159769896...,sizce tareminin son dakikada ki pozisyonu pena...
2,"25/11/2022 20:42:05,000000000",Anti İngiltereciler beğensin. Sayımızı bilelim...,Riberio,tr,9985,3209,0,3,https://twitter.com/riberiioo/status/159624284...,anti i̇ngiltereciler beğensin sayımızı bilelim
3,"28/11/2022 20:54:26,000000000",Torreirasız maç kazanmayı bekliyor adam. Bu ma...,Riberio,tr,9985,3209,0,0,https://twitter.com/riberiioo/status/159733311...,torreirasız maç kazanmayı bekliyor adam bu maç...
4,"05/12/2022 13:44:16,000000000",@ukarakullukcu Bu adam kim?,Riberio,tr,9985,3209,0,0,https://twitter.com/riberiioo/status/159976157...,bu adam kim


In [3]:
df['CLEANED_TEXT'] = df['CLEANED_TEXT'].fillna('').astype(str)
